<a href="https://colab.research.google.com/github/meekmarcelin/transfer_learning_assignment/blob/main/transfer_learning_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()  # Choose the kaggle.json file


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"marcelinm","key":"6240343bb1a4046568e19b66289fccb9"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!pip install kaggle


In [ ]:
!kaggle datasets download -d kritikseth/fruit-and-vegetable-image-recognition


Dataset URL: https://www.kaggle.com/datasets/kritikseth/fruit-and-vegetable-image-recognition
License(s): CC0-1.0
100% 1.97G/1.98G [00:32<00:00, 147MB/s]
100% 1.98G/1.98G [00:32<00:00, 65.2MB/s]


In [ ]:
!unzip fruit-and-vegetable-image-recognition.zip -d fruit-and-vegetable-image-recognition


Archive:  fruit-and-vegetable-image-recognition.zip
  inflating: fruit-and-vegetable-image-recognition/test/apple/Image_1.jpg  
  inflating: fruit-and-vegetable-image-recognition/test/apple/Image_10.jpg  
  inflating: fruit-and-vegetable-image-recognition/test/apple/Image_2.jpg  
  inflating: fruit-and-vegetable-image-recognition/test/apple/Image_3.jpg  
  inflating: fruit-and-vegetable-image-recognition/test/apple/Image_4.jpg  
  inflating: fruit-and-vegetable-image-recognition/test/apple/Image_5.jpg  
  inflating: fruit-and-vegetable-image-recognition/test/apple/Image_6.JPG  
  inflating: fruit-and-vegetable-image-recognition/test/apple/Image_7.jpg  
  inflating: fruit-and-vegetable-image-recognition/test/apple/Image_8.jpg  
  inflating: fruit-and-vegetable-image-recognition/test/apple/Image_9.jpg  
  inflating: fruit-and-vegetable-image-recognition/test/banana/Image_1.jpg  
  inflating: fruit-and-vegetable-image-recognition/test/banana/Image_10.jpg  
  inflating: fruit-and-vegetable

Import Libraries

In [ ]:
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np



In [ ]:
# Parameters
batch_size = 32
epochs = 10
learning_rate = 0.001


 Load Pre-trained Models

In [ ]:
# Parameters
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load pre-trained models
base_models = [VGG16, ResNet50, InceptionV3]
models = []


In [ ]:
for base_model in base_models:
    base = base_model(weights='imagenet', include_top=False)
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base.input, outputs=predictions)
    models.append(model)

87910968/87910968 [==============================] - 1s 0us/step


Freeze Pre-trained Layers

In [ ]:
# Freeze pre-trained layers
for model in models:
    for layer in model.layers[:-2]:
        layer.trainable = False


 Compile Models

In [ ]:
# Compile models
for model in models:
    model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to the dataset
dataset_path = '/content/fruit-and-vegetable-image-recognition'  # Update the dataset path

# Parameters
batch_size = 32  # Number of images per batch
img_size = (224, 224)  # Target size for resizing images

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=10,  # Randomly rotate images by up to 10 degrees
    width_shift_range=0.1,  # Randomly shift images horizontally by up to 10% of the width
    height_shift_range=0.1,  # Randomly shift images vertically by up to 10% of the height
    horizontal_flip=True,  # Randomly flip images horizontally
    validation_split=0.2  # Fraction of images to reserve for validation
)

# Create the training data generator
train_generator = train_datagen.flow_from_directory(
    dataset_path,  # Path to the directory containing the dataset
    subset='training',  # Subset of data (training or validation)
    target_size=img_size,  # Resize images to this size
    batch_size=batch_size,  # Number of images per batch
    class_mode='categorical',  # Type of label (categorical for one-hot encoded labels)
    shuffle=True  # Shuffle the data randomly
)

# Create the validation data generator
validation_generator = train_datagen.flow_from_directory(
    dataset_path,  # Path to the directory containing the dataset
    subset='validation',  # Subset of data (training or validation)
    target_size=img_size,  # Resize images to this size
    batch_size=batch_size,  # Number of images per batch
    class_mode='categorical',  # Type of label (categorical for one-hot encoded labels)
    shuffle=False  # Do not shuffle the data for validation
)


Found 3061 images belonging to 3 classes.
Found 764 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to the dataset
dataset_path = '/content/fruit-and-vegetable-image-recognition'  # Update with your dataset path

# Parameters
batch_size = 32
img_size = (224, 224)

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

# Create the training data generator
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    subset='training',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Create the validation data generator
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    subset='validation',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 3061 images belonging to 3 classes.
Found 764 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load pre-trained models
base_models = [VGG16, ResNet50, InceptionV3]
models = []

for base_model in base_models:
    base = base_model(weights='imagenet', include_top=False)
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)
    model = Model(inputs=base.input, outputs=predictions)

    # Freeze pre-trained layers
    for layer in base.layers:
        layer.trainable = False

    # Compile model
    model.compile(optimizer=Adam(lr=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    models.append(model)


In [ ]:
# Parameters
epochs = 10

# Train the models
for i, model in enumerate(models):
    print(f'Training {base_models[i].__name__} model...')
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator
    )
    print(f'{base_models[i].__name__} model trained successfully.')


Training VGG16 model...
Epoch 1/10
 7/96 [=>............................] - ETA: 31:55 - loss: 0.8731 - accuracy: 0.6964

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


74/96 [======================>.......] - ETA: 8:05 - loss: 0.6702 - accuracy: 0.8010

Evaluate models

In [ ]:
# Evaluate the models
evaluation_results = []
for i, model in enumerate(models):
    print(f'Evaluating {base_models[i].__name__} model...')
    results = model.evaluate(validation_generator)
    evaluation_results.append(results)
    print(f'{base_models[i].__name__} model - Loss: {results[0]}, Accuracy: {results[1]}')


Make Predictions

In [ ]:
# Predictions using the first model as an example
model = models[0]  # Example: Using the first model
print(f'Making predictions using {base_models[0].__name__} model...')
y_pred = model.predict(validation_generator, steps=len(validation_generator))
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels from the generator
true_labels = validation_generator.classes
class_indices = validation_generator.class_indices

# Convert class indices to class labels
class_labels = {v: k for k, v in class_indices.items()}
true_labels = [class_labels[idx] for idx in true_labels]

# Convert predicted class indices to class labels
predicted_labels = [class_labels[idx] for idx in y_pred_classes]

# Display predictions and true labels (first 10 for brevity)
print("Predicted Labels: ", predicted_labels[:10])
print("True Labels: ", true_labels[:10])


Save the Trained Models

In [ ]:
# Save the models
for i, model in enumerate(models):
    model.save(f'{base_models[i].__name__}_model.h5')
    print(f'{base_models[i].__name__} model saved successfully.')


Data Preparation:

Configured Kaggle API in Google Colab to download and unzip the fruit-and-vegetable-image-recognition dataset.
Prepared training and validation data generators with data augmentation techniques (rotation, shift, and horizontal flip).
Model Setup:

Imported necessary libraries and set parameters for batch size, epochs, and learning rate.
Loaded pre-trained models (VGG16, ResNet50, InceptionV3) with ImageNet weights.
Fine-Tuning:

Added custom top layers (GlobalAveragePooling2D and Dense layers).
Froze the pre-trained layers to retain learned features and trained only the new top layers.
Training and Evaluation:

Trained the models using the training data generator.
Evaluated the models on the validation data generator.
Saved the trained models.
Prediction:

Made predictions using one of the trained models to verify its performance on the validation set, demonstrating successful transfer learning with fine-tuning of the top layers.
